In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#import libiraries
import pandas as pd
import numpy as np

#read train and test data
data=pd.read_csv('/content/drive/MyDrive/comptition3/x_train.csv',index_col='id')
test_data=pd.read_csv('/content/drive/MyDrive/comptition3/x_test.csv')

In [3]:
data.head(10)

,text,label
id,,
265723,A group of friends began to volunteer at a hom...,0
284269,British Prime Minister @Theresa_May on Nerve A...,0
207715,"In 1961, Goodyear released a kit that allows P...",0
551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
117912,"In the 1920鈥檚, Hitler was forbidden to address...",0
213064,Nerd Wins Scrabble with word you've never hear...,0
398923,Why 95.8% of Female Newscasters Have the Same ...,1
314798,Donald Trump Says He'll Do This If More 'Inapp...,0


In [4]:
test_data.head(10)

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from Y...
3,3,"As Trump Accuses Iran, He Has One Problem: His..."
4,4,"""Believers"" - Hezbollah 2011"
5,5,The rise of Italian fascism (circa 1925)
6,6,"Trump鈥檚 pick to lead ICE, who touted surge in ..."
7,7,My friend mid-flip
8,8,Look at this cool iPhone case I got the other day
9,9,My rope flying off a friend swing


In [5]:
#checking of the no. of nulls in each columns
data.isnull().sum().sort_values(ascending=False)


text     0
label    0
dtype: int64

In [6]:
#display data information
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 265723 to 34509
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    60000 non-null  object
 1   label   60000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [7]:
#check if there is duplicate in data
data.duplicated().sum()


345

## preprocessing data

In [8]:
# Drop duplicate rows
data.drop_duplicates(subset='text', inplace=True)


In [9]:
#replace label 2 with label 1
data["new_label"] = 0
data.loc[data["label"] == 0, "new_label"] = 0
data.loc[data["label"] > 0, "new_label"] = 1

#checking for the distribution of grade_bad
data['new_label'].value_counts()

0    31949
1    27696
Name: new_label, dtype: int64

In [10]:
#display new data shape
data.shape


(59645, 3)

In [11]:

from bokeh.io import output_notebook
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import pickle
import sklearn
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()
from pathlib import Path
from textblob import TextBlob
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
nltk.download('punkt')
nltk.download('stopwords')

#use lemmatizer to convert the word to root word
#use stop words to remove words that not add much value to the meaning of text
lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words("english"))


def clean_text(text, for_embedding=False):

    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
    #use textblob to correct spelling mistakes
    #m = TextBlob(text)
    #text=m.correct()

    #divide rows into sentences(tokens) to be easier to understod
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no lemmatizing, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            lemmatizer.lemmatize(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
data.text.describe()


count                                                 59645
unique                                                59645
top       A group of friends began to volunteer at a hom...
freq                                                      1
Name: text, dtype: object

In [14]:
#check if train data balanced or not
data["new_label"].value_counts(normalize=True)

0    0.535653
1    0.464347
Name: new_label, dtype: float64

In [15]:
#split columns
test=test_data['text']
x=data['text']
y=data['new_label']
x,y,test

(id
 265723    A group of friends began to volunteer at a hom...
 284269    British Prime Minister @Theresa_May on Nerve A...
 207715    In 1961, Goodyear released a kit that allows P...
 551106    Happy Birthday, Bob Barker! The Price Is Right...
 8584      Obama to Nation: 聙"Innocent Cops and Unarmed Y...
                                 ...                        
 70046     Finish Sniper Simo H盲yh盲 during the invasion o...
 189377    Nigerian Prince Scam took $110K from Kansas ma...
 93486     Is It Safe To Smoke Marijuana During Pregnancy...
 140950    Julius Caesar upon realizing that everyone in ...
 34509     Jeff Bridges Releasing 鈥楽leeping Tapes,鈥?a New...
 Name: text, Length: 59645, dtype: object,
 id
 265723    0
 284269    0
 207715    0
 551106    0
 8584      0
          ..
 70046     0
 189377    1
 93486     0
 140950    0
 34509     1
 Name: new_label, Length: 59645, dtype: int64,
 0                                               stargazer 
 1                          

##Models

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit

###Attempt 1


In [18]:
# Split the original dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(x, y, train_size=0.8, stratify=y, random_state=2022)

# Create a list of indices for each data point indicating whether it belongs to the training or validation set
split_index = [-1 if i in X_train.index else 0 for i in x.index]

# Use the list to create a PredefinedSplit object for GridSearchCV
pds = PredefinedSplit(test_fold=split_index)

# Define the pipeline with TfidfVectorizer and LogisticRegression
pipe = Pipeline([
    ('cvec', TfidfVectorizer(preprocessor=clean_text, analyzer="word", max_df=0.3, min_df=10, norm="l2")),
    ('lr', LogisticRegression(solver='sag'))
])

# Define the hyperparameters to tune using GridSearchCV
pipe_params = {
    'cvec__ngram_range': [(1,1), (2,2), (1,3)],
    'lr__C': [0.01, 0.1, 1],
    'lr__penalty': ['l1', 'l2']
}

# Create a GridSearchCV object with the pipeline and hyperparameters
gs = GridSearchCV(pipe, param_grid=pipe_params, scoring="roc_auc", cv=pds)

# Train the model with GridSearchCV using the entire dataset
gs.fit(x, y)

# Print the best train score and the best parameters found by GridSearchCV
print("Train score:", gs.score(x, y))
print("Best parameters:", gs.best_params_)


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
9 fits failed out of a total of 18.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File 

Train score 0.920156544987508
Best params: {'cvec__ngram_range': (1, 3), 'lr__C': 1, 'lr__penalty': 'l2'}


In [19]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_data['id']
submission['label']=gs.predict_proba(test)[:,1]
submission.to_csv('Logistic_Reg.csv', index=False)

### Attempt 2

In [20]:
# Random Forest pipeline setup
rf_pipe = Pipeline([
    ('tvec', TfidfVectorizer(preprocessor=clean_text, analyzer="char", max_df=0.3, min_df=10, norm="l2")),
    ('rf', RandomForestClassifier())
])

# Setting up Random Forest hyperparameters
rf_params = {
    'tvec__max_features': [2000],
    'tvec__ngram_range': [(1, 2)],
    'tvec__stop_words': ['english'],
    'rf__max_depth': [1000],
    'rf__min_samples_split': [100],
    'rf__max_leaf_nodes': [None]
}

# Fit Random Forest pipeline to the data
rf_pipe.fit(x, y)

# Setting up GridSearch for TFIDFVectorizer
rf_gs = GridSearchCV(rf_pipe, param_grid=rf_params, scoring="roc_auc", cv=pds, verbose=1, n_jobs=-1)

# Fitting Random Forest with GridSearchCV
rf_gs.fit(x, y)

# Print train score and best params
print("Train score:", rf_gs.score(x, y))
print("Best params:", rf_gs.best_params_)


Fitting 1 folds for each of 1 candidates, totalling 1 fits


/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(


Train score 0.9803000126899243
Best params: {'rf__max_depth': 1000, 'rf__max_leaf_nodes': None, 'rf__min_samples_split': 100, 'tvec__max_features': 2000, 'tvec__ngram_range': (1, 2), 'tvec__stop_words': 'english'}


In [22]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_data['id']
submission['label']=rf_gs.predict_proba(test)[:,1]
submission.to_csv('Rand_Forest.csv', index=False)
     

##Attempt 3

In [23]:
# import necessary libraries
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# combine the preprocessor with the model as a full tunable pipeline
full_pipeline = Pipeline(steps=[
    ('cvec', TfidfVectorizer(
        preprocessor=clean_text, # text cleaning function
        analyzer="word", # analyze at the word level
        max_df=0.3, # ignore words that occur in more than 30% of the documents
        min_df=10, # ignore words that occur in less than 10 documents
        norm="l2" # normalize the vectors using the l2 norm
    )),
    ('my_classifier', XGBClassifier(
        n_estimators=200, # number of trees in the forest
        colsample_bytree=0.8, # subsampling ratio of columns when constructing each tree
        subsample=0.8, # subsampling ratio of the training instances
        nthread=10, # number of parallel threads used to run XGBoost
        learning_rate=0.1 # shrinkage factor to prevent overfitting
    ))
])

# fit the pipeline on the training data
full_pipeline.fit(x, y)

# define the hyperparameter grid
param_grid = {
    'cvec__ngram_range': [(1,1), (2,2), (1,3)], # range of n-gram sizes
    'my_classifier__max_depth': [5, 10, 20, 30, 40, 50] # maximum depth of each tree
}

# perform grid search cross-validation to find the best hyperparameters
grid_search = GridSearchCV(
    full_pipeline, # pipeline object
    param_grid=param_grid, # hyperparameter grid
    scoring="roc_auc", # use ROC AUC as the performance metric
    cv=pds, # use the PredefinedSplit object to split the data
    verbose=1, # print progress messages
    n_jobs=-1 # use all available CPUs for parallel processing
)
grid_search.fit(x, y)

# print the best cross-validation score and hyperparameters
print('best score {}'.format(grid_search.best_score_))
print('best params {}'.format(grid_search.best_params_))


Fitting 1 folds for each of 18 candidates, totalling 18 fits
best score 0.8551031369255028
best score {'cvec__ngram_range': (1, 3), 'my_classifier__max_depth': 50}


In [24]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_data['id']
submission['label']=grid_search.predict_proba(test)[:,1]
submission.to_csv('XGB_grid.csv', index=False)

### Attempt 4 


In [25]:
# Define a pipeline that consists of a TfidfVectorizer and an MLPClassifier
full_pipline_5 = Pipeline(steps=[('cvec', TfidfVectorizer(preprocessor=clean_text, analyzer="word", max_df=0.3, min_df=10, norm="l2")), 
                                 ('my_classifier', MLPClassifier(random_state=1, solver="adam", hidden_layer_sizes=(224, 120, 12,), activation="relu", n_iter_no_change=10))])

# Set the hyperparameters for the TfidfVectorizer
param_grid_5 = {'cvec__ngram_range': [(1,1), (2,2), (1,3)]}

# Perform a grid search using the pipeline and hyperparameters defined above
grid_mlp = GridSearchCV(full_pipline_5, param_grid_5, scoring="roc_auc", cv=pds, verbose=1, n_jobs=-1)
grid_mlp.fit(x, y)

# Print the best score and hyperparameters found by the grid search
print('best score {}'.format(grid_mlp.best_score_))
print('best params {}'.format(grid_mlp.best_params_))


Fitting 1 folds for each of 3 candidates, totalling 3 fits
best score 0.8509244449869061
best score {'cvec__ngram_range': (1, 1)}


In [26]:

#predict output and save submission
submission = pd.DataFrame()

submission['id'] = test_data['id']
submission['label']=grid_mlp.predict_proba(test)[:,1]
submission.to_csv('MLP_Grid.csv', index=False)

### Attempt 5

In [27]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

# set vectorizer and classifier hyperparameters as a pipeline
pipeline = Pipeline([
    ('cvec', TfidfVectorizer(preprocessor=clean_text,analyzer="word", max_df=0.3, min_df=10, norm="l2")),    
    ('nb', MultinomialNB())
])

# define hyperparameters grid to search over
param_grid = {
    'cvec__ngram_range': [(1,1), (2,2), (1,3)],
    'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# perform grid search with cross validation
grid_search = GridSearchCV(
    estimator=pipeline, 
    param_grid=param_grid, 
    scoring="roc_auc", 
    cv=pds
)

# train model with grid search
grid_search.fit(x, y)

# print best score and hyperparameters
print("Train score:", grid_search.score(x, y))
print("Best hyperparameters:", grid_search.best_params_)


Train score 0.8961851993624514
Best params: {'cvec__ngram_range': (1, 3), 'nb__alpha': 1}


In [28]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_data['id']
submission['label']=grid_search.predict_proba(test)[:,1]
submission.to_csv('MultinomialNB_grid.csv', index=False)

##**Questions**


### What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?


Character n-grams break down text into sequences of n consecutive characters while word n-grams break down text into sequences of n consecutive words. Character n-grams tend to suffer more from the OOV issue due to generating a larger number of features, especially when using higher values of n. Word n-grams benefit from the fact that words are more semantically meaningful than characters and can capture the overall meaning of the text.

###What is the difference between stop word removal and stemming? Are these techniques language-dependent?


Stop word removal and stemming are both techniques used in natural language processing, but they serve different purposes.

Stop word removal involves removing common words that do not carry much meaning in a sentence, such as "the", "a", "an", etc. These words are often filtered out before processing the text to reduce the computational cost and focus on the more meaningful words in the text.

Stemming involves reducing words to their base or root form, by removing suffixes or prefixes, so that similar words can be treated as the same word. For example, the words "jump", "jumping", and "jumped" would be stemmed to "jump". Stemming is often used to reduce the dimensionality of the text data and improve the efficiency and effectiveness of downstream processing tasks such as information retrieval or text classification.

Both stop word removal and stemming are language-dependent, as the stop words and rules for stemming may differ between languages. For example, the stop words in English may be different from the stop words in French or Arabic, and the rules for stemming English words may be different from those for stemming Arabic words.

## Is tokenization techniques language dependent? Why?

Tokenization techniques are language-dependent because different languages have different rules for dividing text into tokens. For example, in English, words are typically separated by spaces, so tokenization can be done by splitting the text on spaces. However, in languages like Chinese or Japanese, there are no spaces between words, so tokenization requires more advanced techniques such as using language-specific dictionaries or statistical models to identify word boundaries. Additionally, some languages have complex writing systems, such as Arabic or Hebrew, which require special techniques to identify individual words or tokens. Therefore, the choice of tokenization technique depends on the specific language and the characteristics of its writing system, and different techniques may need to be applied to different languages to achieve optimal results.


## What is the difference between count vectorizer and TF-IDF vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?


Count vectorizer represents each document as a vector of word frequencies, while TF-IDF vectorizer assigns a weight to each feature based on its importance in the corpus. It would not be feasible to use all possible n-grams due to the exponential growth of dimensions. To select the optimal n-gram length, one can experiment with different values and evaluate their performance on a validation set or through cross-validation, considering the specific task and text data characteristics.